# Bayesian Optimization
https://github.com/fmfn/BayesianOptimization

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')  # 不打印警告
%matplotlib inline

# 基本用法

#### 安装
pip install bayesian-optimization

In [2]:
from bayes_opt import BayesianOptimization

#### 目标函数
$ f(x, y) = 1 - x^2 - (y - 1)^2 $

In [3]:
f = lambda x, y: 1 - x ** 2 - (y - 1) ** 2

#### 参数范围

In [4]:
pbounds = {'x': (-1, 4), 'y': (-3, 3)}  # Bounded region of parameter space

#### 创建优化器

In [5]:
optimizer = BayesianOptimization(f=f, pbounds=pbounds)

#### 优化

In [6]:
optimizer.maximize(init_points=3, n_iter=7)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -12.42    |  3.473    |  2.166    |
|  2        | -6.326    |  2.552    |  0.09898  |
|  3        | -3.167    |  2.032    |  0.801    |
|  4        | -2.754    |  1.9      |  0.6212   |
|  5        | -1.993    |  1.717    |  0.7872   |
|  6        | -1.289    |  1.474    |  0.6614   |
|  7        | -0.4572   |  1.198    |  0.8536   |
|  8        |  0.2396   |  0.8247   |  0.7167   |
|  9        |  0.7526   |  0.4955   |  1.044    |
|  10       |  0.9504   |  0.06238  |  0.7863   |


#### 结果

In [7]:
optimizer.res

[{'target': -12.424507752814025,
  'params': {'x': 3.473381983379655, 'y': 2.1662441212489085}},
 {'target': -6.325879627577446,
  'params': {'x': 2.5522609732566512, 'y': 0.09897638656444885}},
 {'target': -3.1673181574650227,
  'params': {'x': 2.031679275354415, 'y': 0.8010088455222633}},
 {'target': -2.753730951498059,
  'params': {'x': 1.9000679292636926, 'y': 0.6212219440336568}},
 {'target': -1.9927674788639687,
  'params': {'x': 1.7168184591075526, 'y': 0.7871576702543933}},
 {'target': -1.2885134239882254,
  'params': {'x': 1.474408254666627, 'y': 0.6614239781686598}},
 {'target': -0.4571749329020343,
  'params': {'x': 1.1982192065174164, 'y': 0.8535566115018938}},
 {'target': 0.239643660045436,
  'params': {'x': 0.8246929302447152, 'y': 0.7167370289661049}},
 {'target': 0.7526118466639338,
  'params': {'x': 0.4954534162613094, 'y': 1.0437500360121434}},
 {'target': 0.9504485354889007,
  'params': {'x': 0.062381868231868344, 'y': 0.7863180704247524}}]

In [8]:
optimizer.max

{'target': 0.9504485354889007,
 'params': {'x': 0.062381868231868344, 'y': 0.7863180704247524}}

In [9]:
optimizer.space.target

array([-12.42450775,  -6.32587963,  -3.16731816,  -2.75373095,
        -1.99276748,  -1.28851342,  -0.45717493,   0.23964366,
         0.75261185,   0.95044854])

In [10]:
optimizer.space.target.argmax()

9

# sklearn示例
https://github.com/fmfn/BayesianOptimization/blob/master/examples/sklearn_example.py

In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from bayes_opt import BayesianOptimization


def get_data():
    data, targets = make_classification(
        n_samples=1000,
        n_features=45,
        n_informative=12,
        n_redundant=7,
        random_state=134985745,
    )
    return data, targets


def rfc_cv(n_estimators, min_samples_split, max_features, data, targets):
    estimator = RFC(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=2
    )
    cval = cross_val_score(estimator, data, targets, scoring='neg_log_loss', cv=4)
    return cval.mean()


def optimize_rfc(data, targets):
    def rfc_crossval(n_estimators, min_samples_split, max_features):
        return rfc_cv(
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=max(min(max_features, 0.999), 1e-3),
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=rfc_crossval,
        pbounds={
            "n_estimators": (10, 250),
            "min_samples_split": (2, 25),
            "max_features": (0.1, 0.999),
        },
        random_state=1234,
    )
    optimizer.maximize(n_iter=10)

    print("Final result:", optimizer.max)


data, targets = get_data()
optimize_rfc(data, targets)

|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        | -0.3773   |  0.2722   |  16.31    |  115.1    |
|  2        | -0.3654   |  0.806    |  19.94    |  75.42    |
|  3        | -0.3756   |  0.3485   |  20.44    |  240.0    |
|  4        | -0.354    |  0.8875   |  10.23    |  130.2    |
|  5        | -0.3648   |  0.7144   |  18.39    |  98.86    |
|  6        | -0.582    |  0.999    |  2.0      |  10.0     |
|  7        | -0.3516   |  0.999    |  2.0      |  250.0    |
|  8        | -0.3518   |  0.999    |  2.0      |  197.9    |
|  9        | -0.4275   |  0.1      |  2.0      |  80.14    |
|  10       | -0.3753   |  0.999    |  25.0     |  166.8    |
|  11       | -0.3535   |  0.9979   |  2.031    |  160.1    |
|  12       | -0.4743   |  0.999    |  25.0     |  10.0     |
|  13       | -0.3506   |  0.9604   |  2.071    |  225.8    |
|  14       | -0.3733   |  0.995    |  24.13    |  205.8    |
|  15   

# 返回最优已训练模型

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')  # 不打印警告
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from bayes_opt import BayesianOptimization
import numpy as np
np.random.seed(0)


ls_of_clf = []


def load_data():
    X, y = make_circles(n_samples=5000, noise=.3, factor=.4)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.1)
    return X_train, y_train, X_valid, y_valid, X_test, y_test


def score(X_train, y_train, X_valid, y_valid, C):
    C = 10 ** C  # 幂
    clf = SVC(C).fit(X_train, y_train)
    ls_of_clf.append(clf)
    return clf.score(X_valid, y_valid)


def best_clf(X_train, y_train, X_valid, y_valid):
    f = lambda C: score(X_train, y_train, X_valid, y_valid, C)
    pbounds = {'C': (-3, 2)}  # 取对数使均匀采样【不用(1e-3, 1e2)】
    optimizer = BayesianOptimization(f, pbounds)
    optimizer.maximize(n_iter=10)
    print('Best result:', optimizer.max)
    idx = optimizer.space.target.argmax()
    return ls_of_clf[idx]


X_train, y_train, X_valid, y_valid, X_test, y_test = load_data()
clf = best_clf(X_train, y_train, X_valid, y_valid)

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.7675   | -2.31     |
|  2        |  0.79     |  1.51     |
|  3        |  0.7875   |  1.847    |
|  4        |  0.8      | -2.422    |
|  5        |  0.79     |  1.721    |
|  6        |  0.525    | -3.0      |
|  7        |  0.7925   | -0.2345   |
|  8        |  0.795    |  0.4802   |
|  9        |  0.795    | -1.022    |
|  10       |  0.795    | -0.6551   |
|  11       |  0.795    |  0.9339   |
|  12       |  0.7925   |  0.1263   |
|  13       |  0.7975   | -1.419    |
|  14       |  0.66     | -2.59     |
|  15       |  0.8025   | -1.733    |
Best result: {'target': 0.8025, 'params': {'C': -1.7328121458932235}}


In [2]:
print(clf.score(X_train, y_train))
print(clf.score(X_valid, y_valid))
print(clf.score(X_test, y_test))

0.8191666666666667
0.8025
0.841


In [3]:
clf = SVC().fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_valid, y_valid))
print(clf.score(X_test, y_test))

0.8197222222222222
0.795
0.832


In [4]:
X_train = np.concatenate((X_train, X_valid), axis=0)
y_train = np.concatenate((y_train, y_valid), axis=0)
clf = SVC().fit(X_train, y_train)
print(clf.score(X_train, y_train))
print()
print(clf.score(X_test, y_test))

0.8185

0.835
